# 시흥시 (1193)
- 작성자 : 박선빈
- 작성일자 : 2020년 11월 1일

In [1]:
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd
from pathlib import Path
import numpy as np
import re
from tqdm import tqdm

### 1. Raw File 불러오기

In [2]:
dat ="2020-11-01" 
df = pd.read_csv('201022_type1.csv')
soup = BeautifulSoup(df[df.num == 1193].html.iloc[0], 'html.parser')

### 2. 간단한 전처리 하기

In [3]:
rows = soup.select('ul.info_s')
rows1 = soup.select('ul.list_ul')

### 3. 저장할 DataFrame 미리 만들기

In [5]:
pat_col = ['num', 'dat', 'pth', 'syy', 'syd', 'cnd', 'rsd', 'atn', 'dch', 'adt'] 
rou_col = ['num', 'ord', 'dat', 'rgl', 'rgt', 'frm', 'exd', 'msk', 'ste', 'mob']

patient = pd.DataFrame(columns = pat_col)
route = pd.DataFrame(columns = rou_col)

### 4. 귀찮은 전처리해주기

In [6]:
for i in tqdm(range(len(rows))):
    per=rows[i].select("span");
    rou=rows1[i].select("li"); rou1=[x.text for x in rou if rou[1].text != x.text]; 
    if '\xa0' in rou1:
        rou1.remove('\xa0')
    
    if '시흥' in str(per[0]):
        number= per[0].text.split('#')[1]
        num = '1189' + '0' * (5- len(re.findall("\d+", number)[0])) + re.findall("\d+", number)[0]
        pth=per[2].text; cnd=per[3].text; atn=per[4].text;
        syy='';syd=''; rsd=''; dch=0; adt='';
        if '퇴원' in per[4].text:
            dch = 1;
            
        dt= [rou.index(x) for x in rou if '*' in x.text]
        if len(dt) != 0:
            a=[rou[y].text.split('*')[1] for y in dt]
            adt=", ".join(a)
        
        new_pat = pd.DataFrame([(num, dat, pth, syy, syd, cnd, rsd, atn, dch, adt)], columns = pat_col)
        patient = pd.concat([patient, new_pat])
              
        ind=[rou1.index(x) for x in rou1 if '○' in str(x)]
        date=[rou1[y].replace("○ ","") for y in ind]
        
        if len(ind) != 0:
            for j in range(len(ind)):
                rgl= ''; frm=''; msk=''; ste=''; mob=''; rgt='';           
                for y in ind:
                    exd=date[j]
                    if y == ind[-1]:
                        ord=ind.index(y)+1
                        fin= [rou1.index(z) for z in rou1 if '위 내용은' in z][0]
                        rgt = [rou1[ind[len(ind)-1]+1+k] for k in range(fin-ind[len(ind)-1]-2)][0]
                        
                        new_route = pd.DataFrame(columns = rou_col, data = [[num, ord, dat, rgl, rgt, frm, exd, '', ste, '']])
                        route = pd.concat([route, new_route])
                    else:
                        ord=ind.index(y)+1
                        rgt= [rou1[y+k+1] for k in range(ind[ind.index(y)+1]-y-1)][0]
                        
                        new_route = pd.DataFrame(columns = rou_col, data = [[num, ord, dat, rgl, rgt, frm, exd, '', ste, '']])
                        route = pd.concat([route, new_route])
                
        else:
            new_route = pd.DataFrame(columns = rou_col, data = [[num, 1, dat, '', '', '', '', '', '', '']])
            route = pd.concat([route, new_route])
        

100%|██████████| 185/185 [00:01<00:00, 183.50it/s]


### 5. 결과 잘 나왔나 확인하기

In [7]:
patient

,num,dat,pth,syy,syd,cnd,rsd,atn,dch,adt
0,118900145,2020-11-01,"시흥#142, #144 확진환자 가족",,,2020-10-20,,고양생활치료센터,0,
0,118900144,2020-11-01,시흥#142 확진환자 가족,,,2020-10-17,,경기도의료원 수원병원,0,
0,118900143,2020-11-01,안양시#203 및 #214 확진환자 접촉,,,2020-10-12,,경기도의료원 이천병원,0,
0,118900142,2020-11-01,해외입국자(파키스탄),,,2020-10-11,,이천생활치료센터,0,"접촉자 중복, 접촉자 중복, 접촉자 3명(가족/음성)"
0,118900141,2020-11-01,도봉구#215 확진환자 접촉,,,2020-10-11,,이천생활치료센터,0,"접촉자 중복, 접촉자 중복, 접촉자 중복, 접촉자 중복, 접촉자 1명, 접촉자 중복..."
...,...,...,...,...,...,...,...,...,...,...
0,118900005,2020-11-01,원인불명,,,2020-02-29,,경기도의료원 안성병원/퇴원(20.03.07.),1,
0,118900004,2020-11-01,대구 방문(2.13~2.15),,,2020-02-28,,경기도의료원 수원병원/퇴원(20.03.26),1,
0,118900003,2020-11-01,중국 광동성에서 입국 (1.31.),,,2020-02-09,,경기도의료원 안성병원/퇴원(20.03.07.),1,
0,118900002,2020-11-01,중국 광동성에서 입국(1.31.),,,2020-02-09,,경기도의료원 안성병원/퇴원(20.03.08.),1,


In [8]:
route

,num,ord,dat,rgl,rgt,frm,exd,msk,ste,mob
0,118900145,1,2020-11-01,,자택 머무름,,10월 17~10월 19일(19일 증상발현),,,
0,118900145,2,2020-11-01,,정왕보건지소 선별진료소 검사,,10월 17~10월 19일(19일 증상발현),,,
0,118900145,3,2020-11-01,,고양생활치료센터 이송,,10월 17~10월 19일(19일 증상발현),,,
0,118900145,1,2020-11-01,,자택 머무름,,10월 20일(확진일),,,
0,118900145,2,2020-11-01,,정왕보건지소 선별진료소 검사,,10월 20일(확진일),,,
...,...,...,...,...,...,...,...,...,...,...
0,118900005,1,2020-11-01,,,,,,,
0,118900004,1,2020-11-01,,,,,,,
0,118900003,1,2020-11-01,,,,,,,
0,118900002,1,2020-11-01,,,,,,,
